In [1]:
import sys
import os

# プロジェクトルートを Python パスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import numpy as np
from graph.prior import GaussianPrior
from graph.measurement import GaussianMeasurement
from core.uncertain_array import UncertainArray as UA

In [3]:
# 1. Prior → X
X = GaussianPrior(shape=(4, 4), mean=0, var=1.0, seed=42)

# 2. X → Measurement（観測値 y）
obs_data = np.ones((4, 4), dtype=np.complex128)
meas = GaussianMeasurement(X, obs_data, var=0.25)

# 3. メッセージ伝播
X.parent.forward()   # Prior → X
X.forward()          # X → Measurement
meas.backward()      # Measurement → X
X.compute_belief()   # Belief 更新

# 4. 結果確認
print("Mean:\n", X.belief.data)
print("Precision:\n", X.belief.precision)

Mean:
 [[0.8430935 +0.05214924j 0.65292404-0.13560648j 0.90612983+0.12423163j
  0.93301594-0.00706059j]
 [0.52408196-0.02614349j 0.61584401-0.09629798j 0.81807936+0.17289345j
  0.75527654-0.02185377j]
 [0.79762396-0.0605747j  0.67936137-0.0497992j  0.92436565+0.07527989j
  0.90999639+0.0516816j ]
 [0.80933815+0.05836921j 0.95941598+0.06092729j 0.86611581+0.30287471j
  0.67847769-0.05747576j]]
Precision:
 [[5. 5. 5. 5.]
 [5. 5. 5. 5.]
 [5. 5. 5. 5.]
 [5. 5. 5. 5.]]


In [4]:
import numpy as np

# ライブラリのインポート
from graph.prior import GaussianPrior
from graph.measurement import MaskedGaussianMeasurement
from core.uncertain_array import UncertainArray as UA
from core.linalg_utils import random_binary_mask

# 1. Prior によって Wave を生成
X = GaussianPrior(shape=(8, 8), mean=0, var=1.0, seed=42)

# 2. ランダムマスクを生成し、観測値を作る（全て 1+0j にする）
mask = random_binary_mask((8, 8), subsampling_rate=0.5, seed=1)
obs_data = np.ones((8, 8), dtype=np.complex128)

# 3. マスク付き GaussianMeasurement を構築
meas = MaskedGaussianMeasurement(X, observed_data=obs_data, var=0.25, mask=mask)

# 4. メッセージ伝播を実行
X.parent.forward()   # Prior → X
X.forward()          # X → Measurement
meas.backward()      # Measurement → X
X.compute_belief()   # Belief 更新

# 5. 結果表示
print("Mask (True = observed):\n", mask.astype(int))
print("Belief precision:\n", X.belief.precision)
print("Belief mean:\n", X.belief.data)


Mask (True = observed):
 [[0 1 0 1 0 1 0 1]
 [0 1 1 1 0 1 0 0]
 [1 1 0 1 0 0 1 1]
 [1 1 1 1 0 0 1 1]
 [0 1 1 1 1 1 0 0]
 [0 0 0 0 1 0 0 1]
 [0 1 1 0 1 1 0 0]
 [0 1 0 1 0 0 0 0]]
Belief precision:
 [[1. 5. 1. 5. 1. 5. 1. 5.]
 [1. 5. 5. 5. 1. 5. 1. 1.]
 [5. 5. 1. 5. 1. 1. 5. 5.]
 [5. 5. 5. 5. 1. 1. 5. 5.]
 [1. 5. 5. 5. 5. 5. 1. 1.]
 [1. 1. 1. 1. 5. 1. 1. 5.]
 [1. 5. 5. 1. 5. 5. 1. 1.]
 [1. 5. 1. 5. 1. 1. 1. 1.]]
Belief mean:
 [[ 0.21546751+0.50291314j  0.65292404+0.11219624j  0.53064913-0.24658586j
   0.93301594-0.06538642j -1.37959021+0.60668056j  0.61584401-0.02705452j
   0.09039682-0.90204645j  0.75527654-0.16027101j]
 [-0.01188021-0.65015095j  0.67936137+0.07030915j  0.92436565+0.02014204j
   0.90999639+0.09764938j  0.04669075-0.30211324j  0.95941598+0.0224209j
   0.33057903+0.44235921j -0.60761153-0.21874104j]
 [ 0.85214924+0.06459777j  0.66439352-0.09361046j  0.62115816-0.25671784j
   0.79293941-0.05398589j -0.13071743-0.84558632j -0.4814899 +0.34434154j
   0.97289345-0.06638352j  